In [58]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

file_path='/home/naseem_fordham/Pytorch/Infusion/llm/data'
# /home/naseem_fordham/Pytorch/data/PV_2class.csv
# Load pre-trained GloVe embeddings


def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding="utf8") as f:
        for line in f:
            values = line.split()
            if not values[0].isalnum():
                # print(f"Skipping line: {line}")
                continue
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            except ValueError:
                print(f"Issue parsing line: {line}")
                continue
    return embeddings_index


# def load_glove_embeddings(file_path):
#     embeddings_index = {}
#     with open(file_path, encoding="utf8") as f:
#         for line in f:
#             values = line.split()
#             word = values[0]
#             try:
#                 coefs = np.asarray(values[1:], dtype='float32')
#                 embeddings_index[word] = coefs
#             except ValueError:
#                 print(f"Issue parsing line: {line}")
#                 continue
#     return embeddings_index

# Load GloVe embeddings
path='/home/naseem_fordham/Pytorch/Infusion/llm/data/'
glove_embeddings = load_glove_embeddings(f'{path}glove.6B.100d.txt')



In [63]:
from sklearn.model_selection import train_test_split

df=pd.read_csv('/home/naseem_fordham/Pytorch/data/PV_2class.csv')

train_dataset, temp_dataset = train_test_split(df, test_size= 0.20, stratify= df["class"], random_state = 47)
val_dataset,   test_dataset   = train_test_split(temp_df, test_size= 0.20, stratify= temp_df["class"],random_state = 47)




In [60]:
# df_train['class'].values

In [67]:




train_texts=train_dataset['tweet'].tolist()
val_texts=val_dataset['tweet'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(train_texts)

# Map words to embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1,100))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Pad sequences
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Define LSTM model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 100, weights=[embedding_matrix], input_length=max_sequence_length, trainable=True),  
    LSTM(128),
    Dense(2, activation='softmax')
])
train_label=train_dataset['class'].values
val_labels=val_dataset['class'].values
# Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(padded_sequences, train_label, epochs=10, validation_split=0.2)


# Evaluate the model



Epoch 1/10
64/64 [==============================] - 2s 23ms/step - loss: 0.6200 - accuracy: 0.6688 - val_loss: 0.6524 - val_accuracy: 0.6230
Epoch 2/10
64/64 [==============================] - 1s 19ms/step - loss: 0.5606 - accuracy: 0.7044 - val_loss: 0.6063 - val_accuracy: 0.6992
Epoch 3/10
64/64 [==============================] - 1s 19ms/step - loss: 0.5353 - accuracy: 0.7230 - val_loss: 0.6075 - val_accuracy: 0.6797
Epoch 4/10
64/64 [==============================] - 1s 19ms/step - loss: 0.5063 - accuracy: 0.7445 - val_loss: 0.6605 - val_accuracy: 0.6270
Epoch 5/10
64/64 [==============================] - 1s 19ms/step - loss: 0.4820 - accuracy: 0.7484 - val_loss: 0.6689 - val_accuracy: 0.6445
Epoch 6/10
64/64 [==============================] - 1s 19ms/step - loss: 0.4506 - accuracy: 0.7694 - val_loss: 0.6788 - val_accuracy: 0.5996
Epoch 7/10
64/64 [==============================] - 1s 19ms/step - loss: 0.4227 - accuracy: 0.7851 - val_loss: 0.8007 - val_accuracy: 0.6016
Epoch 8/10
64

In [68]:
# Prepare test data
test_texts = test_dataset['tweet'].tolist()
test_sequences = tokenizer.texts_to_sequences(test_texts)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)
test_labels = test_dataset['class'].values # Example test labels, adjust according to your dataset


In [69]:
predictions = model.predict(padded_test_sequences)

# Extract softmax values from predictions
softmax_values = np.array([prediction[1] for prediction in predictions]) 


4/4 [==============================] - 0s 8ms/step


In [70]:
prediction=pd.DataFrame(predictions)

In [72]:
prediction.to_csv('/home/naseem_fordham/Pytorch/Infusion/data/llm/lstm_glove.csv')